In [2]:
import pyspark
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
type(sc)

pyspark.context.SparkContext

In [3]:
modelo = [('marca1', 'modelo1', 'transmicion', 'arg'),
          ('marca2', 'modelo2', 'transmicion', 'arg'),
          ('marca3', 'modelo3', 'transmicion', 'arg'),
          ('marca4', 'modelo4', 'transmicion', 'ury'),
          ('marca5', 'modelo5', 'transmicion', 'arg'),
          ('marca6', 'modelo6', 'transmicion', 'arg'),
          ('marca7', 'modelo7', 'transmicion', 'arg'),
         ]

In [54]:
ventas = [('marca1', 'modelo1', 20190521, 'con1'),
          ('marca1', 'modelo1', 20190522, 'con1'),
          ('marca1', 'modelo1', 20180401, 'con1'),
          ('marca4', 'modelo4', 20180901, 'con2'),
          ('marca4', 'modelo4', 20190401, 'con2'),
          ('marca2', 'modelo2', 20180601, 'con2'),
          ('marca2', 'modelo2', 20180528, 'con3'),
          ('marca2', 'modelo2', 20180312, 'con3'),
          ('marca2', 'modelo2', 20180102, 'con3'),
          ('marca3', 'modelo3', 20180219, 'con3'),
        ]

In [55]:
venta = sc.parallelize(ventas)
modelos = sc.parallelize(modelo)

In [13]:
m1 = modelos.filter(lambda x: x[3] == 'arg').map(lambda x: ((x[0], x[1]), 1))
m1.collect()

[(('marca1', 'modelo1'), 1),
 (('marca2', 'modelo2'), 1),
 (('marca3', 'modelo3'), 1),
 (('marca5', 'modelo5'), 1),
 (('marca6', 'modelo6'), 1),
 (('marca7', 'modelo7'), 1)]

In [79]:
v1 = venta.map(lambda x: ((x[0], x[1]), (x[2], x[2])))

In [80]:
join = m1.join(v1)
join.collect()

[(('marca3', 'modelo3'), (1, (20180219, 20180219))),
 (('marca1', 'modelo1'), (1, (20190521, 20190521))),
 (('marca1', 'modelo1'), (1, (20190522, 20190522))),
 (('marca1', 'modelo1'), (1, (20180401, 20180401))),
 (('marca2', 'modelo2'), (1, (20180601, 20180601))),
 (('marca2', 'modelo2'), (1, (20180528, 20180528))),
 (('marca2', 'modelo2'), (1, (20180312, 20180312))),
 (('marca2', 'modelo2'), (1, (20180102, 20180102)))]

In [105]:
def fechas(x,y):
    mayor = x[1][1] if x[1][1] >= y[1][1] else y[1][1]
    menor = x[1][1] if x[1][1] < y[1][1] else y[1][1]
    return (x[0] + y[0], (mayor, menor))

In [106]:
j1 = join.reduceByKey(fechas)
j1.collect()

[(('marca3', 'modelo3'), (1, (20180219, 20180219))),
 (('marca1', 'modelo1'), (3, (20190521, 20180401))),
 (('marca2', 'modelo2'), (4, (20180312, 20180102)))]

In [103]:
j2 = j1.filter(lambda x: x[1][1][0] < 20180614)\
.map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1][1], x[1][1][0]))\
.sortBy(lambda x: x[4])

In [104]:
j2.collect()

[('marca3', 'modelo3', 1, 20180219, 20180219),
 ('marca2', 'modelo2', 4, 20180102, 20180312)]

In [107]:
import pandas

# NBA

In [109]:
datos_jugadores_regtemp = [
    (1, 'Manu Ginobili', 30, 35, 8, 8, 5, 2.2),
    (2, 'Kobe Bryant', 40, 30, 3, 5, 20, 2.1),
    (3, 'LeBron James', 58, 20, 3, 8, 30, 2.9),
    (4, 'Andre Iguodala', 35, 15, 5, 10, 20, 4),
    (5, 'Carmelo Anthony', 40, 10, 6, 6, 22, 1.1)
]

datos_jugadores_po = [
    (1, 1, 1, 8, 3, 4, 2, 12, 1),
    (1, 1, 3, 4, 1, 5, 2, 15, 1),
    (1, 1, 4, 12, 2, 3, 2, 10, 1),
    (2, 1, 1, 18, 3, 3, 3, 8, 2),
    (2, 1, 4, 9, 3, 4, 1, 5, 1),
    (3, 2, 1, 12, 6, 4, 1, 3, 1),
    (3, 2, 2, 15, 8, 5, 3, 7, 1),
    (3, 2, 3, 22, 6, 2, 1, 4, 2),
    (3, 2, 4, 18, 10, 4, 2, 4, 1),
    (4, 2, 1, 12, 2, 1, 1, 9, 0),
    (4, 2, 2, 25, 8, 4, 2, 4, 3),
    (4, 2, 4, 15, 4, 2, 2, 8, 1),
    (1, 3, 3, 2, 1, 0, 0, 3, 2),
    (5, 3, 1, 8, 1, 2, 0, 6, 0),
    (5, 3, 1, 7, 0, 0, 1, 6, 1),
]

tr = sc.parallelize(datos_jugadores_regtemp)
po = sc.parallelize(datos_jugadores_po)

In [111]:
po1 = po.map(lambda x: (x[0], x[7])) # cantidas de asistencias, hay que sumarlas
tr1 = tr.map(lambda x: (x[0], (x[1], x[3]))) # nombre y promedio, son todos iguales
join = po1.join(tr1)

In [112]:
join.collect()

[(1, (12, ('Manu Ginobili', 35))),
 (1, (15, ('Manu Ginobili', 35))),
 (1, (10, ('Manu Ginobili', 35))),
 (1, (3, ('Manu Ginobili', 35))),
 (2, (8, ('Kobe Bryant', 30))),
 (2, (5, ('Kobe Bryant', 30))),
 (3, (3, ('LeBron James', 20))),
 (3, (7, ('LeBron James', 20))),
 (3, (4, ('LeBron James', 20))),
 (3, (4, ('LeBron James', 20))),
 (4, (9, ('Andre Iguodala', 15))),
 (4, (4, ('Andre Iguodala', 15))),
 (4, (8, ('Andre Iguodala', 15))),
 (5, (6, ('Carmelo Anthony', 10))),
 (5, (6, ('Carmelo Anthony', 10)))]

In [122]:
join1 = join.reduceByKey(lambda x, y: (x[0] + y[0], (x[1][0], x[1][1])))\
                         .filter(lambda x: x[1][0] > x[1][1][1])

In [123]:
join1.collect()

[(1, (40, ('Manu Ginobili', 35))),
 (4, (21, ('Andre Iguodala', 15))),
 (5, (12, ('Carmelo Anthony', 10)))]

In [126]:
join1.map(lambda x: x[1][1][0]).collect()

['Manu Ginobili', 'Andre Iguodala', 'Carmelo Anthony']

# Anagrama de mas de 7 letras

In [172]:
soy_un_7 = 2

In [153]:
texto = sc.parallelize(['hola ahlo gato otra agot aloh dada dado duda'])

In [173]:
tex = texto.map(lambda x: x.split())\
            .flatMap(lambda x: x)\
            .filter(lambda x: len(x)> soy_un_7)\
            .map(lambda x: (''.join(sorted(x)), [x]))\
            .reduceByKey(lambda x,y: x+y)\
            .filter(lambda x: len(x[1])>1)\
            .map(lambda x: x[1])\
            .collect()

In [171]:
tex

[['hola', 'ahlo', 'aloh'], ['gato', 'agot']]